<a href="https://colab.research.google.com/github/03axdov/Notebooks/blob/main/KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
pip install -q -U keras-tuner

     |████████████████████████████████| 135 kB 7.5 MB/s 
     |████████████████████████████████| 1.6 MB 45.0 MB/s 


In [3]:
import keras_tuner as kt

In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
train_images = train_images.astype("float32") / 255.0
test_images = test_images.astype("float32") / 255.0

In [6]:
def create_model(hp):
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model = keras.Sequential([
      keras.layers.Flatten(input_shape=(28, 28)),
      keras.layers.Dense(units=hp_units, activation="relu"),
      keras.layers.Dense(10)
  ])

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(
      optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
      loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=["accuracy"]
  )

  return model

In [7]:
tuner = kt.Hyperband(create_model,
                     objective="val_accuracy",
                     max_epochs=10,
                     factor=3,
                     directory="my_dir",
                     project_name="intro_to_kt")

In [8]:
cp = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

In [9]:
tuner.search(train_images, train_labels, epochs=50, validation_split=0.2, callbacks=[cp])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
                                                        
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 19s]
val_accuracy: 0.8832499980926514

Best val_accuracy So Far: 0.8915833234786987
Total elapsed time: 00h 15m 16s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 480 and the optimal learning rate for the optimizer
is 0.001.



In [10]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_images, train_labels, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print(f"Best epoch: {best_epoch}")

Epoch 1/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.4902 - accuracy: 0.8253 - val_loss: 0.4060 - val_accuracy: 0.8522
Epoch 2/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3704 - accuracy: 0.8666 - val_loss: 0.4185 - val_accuracy: 0.8388
Epoch 3/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3286 - accuracy: 0.8789 - val_loss: 0.3384 - val_accuracy: 0.8763
Epoch 4/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3053 - accuracy: 0.8861 - val_loss: 0.3715 - val_accuracy: 0.8608
Epoch 5/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2858 - accuracy: 0.8948 - val_loss: 0.3266 - val_accuracy: 0.8807
Epoch 6/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2693 - accuracy: 0.9011 - val_loss: 0.3445 - val_accuracy: 0.8750
Epoch 7/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2579 - accuracy: 0.9023 - val_loss: 0.3134 - val_accuracy:

In [11]:
loss, acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.5922 - accuracy: 0.8865
